# FPUS23 — YOLO Smart Ensemble (Abdomen+Head) + (Arms+Legs)

- Train two specialists and fuse with Weighted Boxes Fusion (WBF).
- A: Abdomen+Head (AH), 768px, YOLO11s.
- B: Arms+Legs (AL), 896px, YOLO11m, light composition augs.
- Save every epoch to Drive and auto‑resume.


In [ ]:
%pip -q install -U ultralytics ensemble-boxes pyyaml
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
REPO_URL = os.environ.get('FPUS23_REPO_URL', 'https://github.com/Srinivas-Raghav-VC/MultiFetalOrgan-Detection.git')
REPO_DIR = '/content/fpus23'
if not os.path.isdir(REPO_DIR):
    !git clone $REPO_URL $REPO_DIR
else:
    !git -C $REPO_DIR fetch && git -C $REPO_DIR pull --ff-only
%cd /content/fpus23
os.makedirs('/content/drive/MyDrive/FPUS23_runs', exist_ok=True)


In [ ]:
# Use despeckled originals for val/test; train specialists on balanced-despeckled for stability.
from pathlib import Path
import yaml
DESP_YAML = Path('/content/fpus23_project/dataset/fpus23_yolo_despeckled/data.yaml')
DESP_BAL_YAML = Path('/content/fpus23_project/dataset/fpus23_yolo_balanced_despeckled/data.yaml')
assert DESP_YAML.exists(), 'Run the SOTA YOLO notebook first to build despeckled dataset.'
assert DESP_BAL_YAML.exists(), 'Run the SOTA YOLO notebook balance cell first.'
names = yaml.safe_load(DESP_YAML.read_text()).get('names', ['Abdomen','Arms','Legs','Head'])
print('Class names:', names)


In [ ]:
# Train Model-A (Abdomen+Head) on balanced-despeckled
RUN_A='fpus23_yolo_AH_s'
!yolo train data=$DESP_BAL_YAML model=yolo11s.pt epochs=60 batch=16 imgsz=768 \
+       project='/content/drive/MyDrive/FPUS23_runs' name=$RUN_A deterministic=True workers=2 \
+       cos_lr=True cls=3.2 resume=True save_period=1 patience=20 classes=[0,3]


In [ ]:
# Train Model-B (Arms+Legs) with small-object friendly settings
RUN_B='fpus23_yolo_AL_m'
!yolo train data=$DESP_BAL_YAML model=yolo11m.pt epochs=60 batch=8 imgsz=896 \
+       project='/content/drive/MyDrive/FPUS23_runs' name=$RUN_B deterministic=True workers=2 \
+       cos_lr=True cls=3.4 mosaic=0.25 close_mosaic=10 copy_paste=0.08 erasing=0.35 \
+       resume=True save_period=1 patience=25 classes=[1,2]


In [ ]:
# Run WBF ensemble on validation and evaluate COCO mAP
import json, math
from ultralytics import YOLO
from ensemble_boxes import weighted_boxes_fusion
from pathlib import Path
import yaml
# Paths
VAL_DIR = Path(yaml.safe_load(DESP_YAML.read_text())['val'])
ANN = Path('/content/fpus23_project/dataset/fpus23_coco/val.json')
BEST_A = Path('/content/drive/MyDrive/FPUS23_runs')/RUN_A/'weights'/'best.pt'
BEST_B = Path('/content/drive/MyDrive/FPUS23_runs')/RUN_B/'weights'/'best.pt'
assert BEST_A.exists() and BEST_B.exists(), 'Train both models first.'
modelA = YOLO(str(BEST_A))
modelB = YOLO(str(BEST_B))
imgs = sorted([p for p in VAL_DIR.glob('*.png')]+[p for p in VAL_DIR.glob('*.jpg')]+[p for p in VAL_DIR.glob('*.jpeg')])
W, H = 768, 768  # not used for normalization (we use per-image sizes)
def run_pred(m, imlist, imgsz):
    return m.predict(source=imlist, imgsz=imgsz, conf=0.001, iou=0.6, verbose=False, save=False)
resA = run_pred(modelA, imgs, 896)
resB = run_pred(modelB, imgs, 896)
# Build COCO JSON via WBF
with open(ANN, 'r') as f:
    gt = json.load(f)
img_id_by_name = {x['file_name']: x['id'] for x in gt['images']}
cat_id_by_name = {x['name']: x['id'] for x in gt['categories']}
name_by_idx = names
preds = []
for ia, ib in zip(resA, resB):
    assert ia.path == ib.path
    path = Path(ia.path).name
    img_id = img_id_by_name.get(path)
    if img_id is None:
        continue
    # Collect boxes from A and B (normalized xyxy in [0,1])
    def collect(rs, allow):
        if rs.boxes is None or len(rs.boxes)==0:
            return [], [], []
        xyxy = rs.boxes.xyxy.cpu().numpy()
        cls = rs.boxes.cls.cpu().numpy()
        conf = rs.boxes.conf.cpu().numpy()
        w = rs.orig_shape[1]; h = rs.orig_shape[0]
        boxes = []
        labels = []
        scores = []
        for b,(x1,y1,x2,y2),c,s in zip(rs.boxes, xyxy, cls, conf):
            c = int(c)
            if c not in allow:
                continue
            boxes.append([x1/w, y1/h, x2/w, y2/h])
            labels.append(c)
            scores.append(float(s))
        return boxes, scores, labels
    # A only Abdomen(0), Head(3) ; B only Arms(1), Legs(2)
    bA,sA,lA = collect(ia, {0,3})
    bB,sB,lB = collect(ib, {1,2})
    boxes = [bA, bB]
    scores= [sA, sB]
    labels= [lA, lB]
    if len(bA)+len(bB)==0:
        continue
    fb, fs, fl = weighted_boxes_fusion(boxes, scores, labels, weights=[1.0,1.0], iou_thr=0.55, skip_box_thr=0.001)
    # Convert fused back to COCO bbox and append
    for (x1,y1,x2,y2), sc, lab in zip(fb, fs, fl):
        w = ia.orig_shape[1]; h = ia.orig_shape[0]
        x = float(x1*w); y = float(y1*h); ww = float((x2-x1)*w); hh=float((y2-y1)*h)
        cname = name_by_idx[int(lab)]
        cat_id = cat_id_by_name.get(cname, None)
        if cat_id is None:
            continue
        preds.append({'image_id': int(img_id), 'category_id': int(cat_id), 'bbox': [x,y,ww,hh], 'score': float(sc)})
# Write and evaluate
OUT_DIR = Path('/content/drive/MyDrive/FPUS23_runs/fpus23_yolo_smart_ensemble')
OUT_DIR.mkdir(parents=True, exist_ok=True)
PRED_JSON = OUT_DIR/'preds_val.json'
PRED_JSON.write_text(json.dumps(preds))
print('Wrote', PRED_JSON)
!python '/content/fpus23/scripts/eval_generic_coco.py' --gt /content/fpus23_project/dataset/fpus23_coco/val.json --pred $PRED_JSON --save $OUT_DIR/metrics_val.json
